# Oracle efficiency computations

If you had to estimate a metric on a dataset of size $N$ but could only observe the labels of a data subset of size $n$, how should you strategically select this subset to obtain the most precise estimate of the metric?

The goal of this notebook is to measure the precision of our estimates of a metric of interest $\mathbb{E}[Z]$ across sampling methods and estimators on a given dataset (see the paper for details on the notation). We analyze:
* Estimators: Horvitz-Thompson (HT) and difference (DF) estimators
* Sample designs: simple random sampling (SRS) and stratified simple random sampling (SSRS) with proportional and Neyman allocation, all without replacement

For this benchmarking exercise, we will assume that we have access to all labels $Z$. This will allow us to compute the exact risk of the estimators, which otherwise would not be possible. 
We will focus on estimating the precision of the accuracy estimates of a multi-class classifier. Other metrics can be estimated in a similar way. 

### Load the data

Consider a multi-class classification task on Caltech 101. Predictions are generated by a CLIP model with ResNet 50 as visual encoder. Let's load packages as well as predictions $(m_1(X), \dots, m_K(X))$ and ground truth labels ($Y$). You can also plug in your own data! 

In [6]:
import torch
import numpy as np
import pandas as pd
from cascade import OracleEstimator, ModelPerformanceEvaluator

In [7]:
data_folder = "../data/predictions/"
data_name = "caltech101/RN50_zero_shot.pt"
df = torch.load(data_folder + data_name)

labels, preds = np.array(df["Target"]), np.array(df["PredictionProb"])

We now compute the values of the performance metric, $Z$. Here we take $Z$ to be the accuracy of the classifier. 

In [8]:
predicted_labels = np.argmax(preds, axis=1)
performance = (predicted_labels == labels)

### Predict performance

We obtain an estimate of the expected performance for each observation, that is we construct a proxy $\hat{Z}$ for $Z$. This proxy can be based on _anything_, but, the more strongly associated it is with $Z$, the more precise our estimates of $\mathbb{E}[Z]$ will be. 

Here we use the predictions of the model under evaluation to construct $\hat{Z}$. This means that we set $\hat{Z} = \argmax_{k\in [K]} m_k(X)$. Ideally, you would want to at least calibrate these predictions. 

In [9]:
proxy_performance =  preds[torch.arange(len(predicted_labels)), predicted_labels]

### Stratify

SSRS requires dividing the population into strata, from which we will draw the observations. 
We form the strata by running k-means on the predictions, following the recommendations from the paper. However, other methods can be applied here as well. 

In [10]:
oracle = OracleEstimator(performance=performance, proxy_performance=proxy_performance, total_samples=len(proxy_performance), budget=100)

from sklearn.cluster import KMeans
oracle.evaluator.stratify_data(KMeans(n_clusters=10, random_state=0, n_init="auto"), oracle.proxy_performance)

### Calculate the variance of the sample design+estimator


We are now ready to compute the variance of our estimators. 
To compute the variance of the DF estimator instead of HT, we simply need to pass `outcomes-predictions` to the functions above instead of `outcomes`!

In [11]:
def calculate_sampling_variances(oracle):
    
    estimators = {
        "HT": performance,
        "DF": performance - proxy_performance
    }
    
    strata_labels = oracle.evaluator.strata_labels
    
    results = {}
    for est, values in estimators.items():
        results[est] = {}

         
        results[est]['SRS'] = oracle.get_srs_variance(outcomes=values)
        
        oracle.evaluator.allocate_budget(allocation_type="proportional")
        results[est]['SSRS proportional'] = oracle.get_ssrs_proportional_variance(outcomes=values)
        
        oracle.evaluator.allocate_budget(variances_by_strata=[
            np.mean(oracle.proxy_performance[oracle.evaluator.strata_labels == s]) * (1 - np.mean(oracle.proxy_performance[oracle.evaluator.strata_labels == s]))
            for s in np.unique(oracle.evaluator.strata_labels)
        ], allocation_type="neyman")
        results[est]['SSRS Neyman'] = oracle.get_ssrs_neyman_variance(outcomes=values)

    return pd.DataFrame.from_dict(results, orient="index").T  

results = calculate_sampling_variances(oracle)

Let's look at the variances of the estimators under the different sampling designs.  

In [12]:
results

,HT,DF
SRS,0.002398,0.001711
SSRS proportional,0.001611,0.001606
SSRS Neyman,0.002110,0.002095
